In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

In [2]:
train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=(0.1307,),std=(0.3081,))
                                       ])
test_transforms = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=(0.1307,),std=(0.3081,))
])
train = datasets.MNIST(root = "./data",train=True,transform=train_transforms,download=True)
test = datasets.MNIST(root="./data",train=False,transform=test_transforms,download=True)

In [3]:
seed = 1
cuda = torch.cuda.is_available()
torch.manual_seed(seed)
if cuda:
  torch.cuda.manual_seed(seed)
dataloader_args=dict(shuffle=True,batch_size=128,num_workers=4,pin_memory=True) if cuda else dict(shuffle=True,batch_size =64)
train_loader = torch.utils.data.DataLoader(dataset=train,**dataloader_args)
test_loader = torch.utils.data.DataLoader(dataset=test,**dataloader_args)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=0, bias=False)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False)

        self.pool  = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False)
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=0, bias=False)

        self.conv5 = nn.Conv2d(in_channels=32, out_channels=10, kernel_size=(1, 1), padding=0, bias=False)
        self.out = nn.AdaptiveAvgPool2d(1)

    def forward(self,x):

      x = F.relu(self.conv1(x))
      x = F.relu(self.conv2(x))

      x = self.pool(x)

      x = F.relu(self.conv3(x))
      x = F.relu(self.conv4(x))

      x = F.relu(self.conv5(x))

      x = self.out(x)
      x = x.view(-1, 10)

      return F.log_softmax(x, dim =1)

In [5]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
            Conv2d-2           [-1, 16, 24, 24]           2,304
         MaxPool2d-3           [-1, 16, 12, 12]               0
            Conv2d-4           [-1, 16, 10, 10]           2,304
            Conv2d-5             [-1, 32, 8, 8]           4,608
            Conv2d-6             [-1, 10, 8, 8]             320
 AdaptiveAvgPool2d-7             [-1, 10, 1, 1]               0
Total params: 9,680
Trainable params: 9,680
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.20
Params size (MB): 0.04
Estimated Total Size (MB): 0.24
----------------------------------------------------------------


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [7]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [8]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Loss=2.3039186000823975 Batch_id=0 Accuracy=7.81:   0%|          | 0/938 [00:00<?, ?it/s]

EPOCH: 0


Loss=0.3851735591888428 Batch_id=937 Accuracy=46.17: 100%|██████████| 938/938 [00:55<00:00, 16.98it/s]
Loss=0.4327898323535919 Batch_id=2 Accuracy=86.98:   0%|          | 2/938 [00:00<00:55, 16.86it/s]


Test set: Average loss: 0.4683, Accuracy: 8417/10000 (84.17%)

EPOCH: 1


Loss=0.08011869341135025 Batch_id=937 Accuracy=91.82: 100%|██████████| 938/938 [00:55<00:00, 16.99it/s]
Loss=0.3242563307285309 Batch_id=2 Accuracy=92.19:   0%|          | 2/938 [00:00<00:55, 16.84it/s] 

...


Test set: Average loss: 0.0601, Accuracy: 9803/10000 (98.03%)

EPOCH: 13


Loss=0.005866040010005236 Batch_id=937 Accuracy=98.59: 100%|██████████| 938/938 [00:55<00:00, 16.85it/s]
Loss=0.032122768461704254 Batch_id=2 Accuracy=98.96:   0%|          | 2/938 [00:00<00:54, 17.04it/s]


Test set: Average loss: 0.0563, Accuracy: 9815/10000 (98.15%)

EPOCH: 14


Loss=0.10056784003973007 Batch_id=937 Accuracy=98.71: 100%|██████████| 938/938 [00:56<00:00, 16.69it/s]



Test set: Average loss: 0.0532, Accuracy: 9827/10000 (98.27%)

